In [8]:
import time
import numpy as np
import numpy as np
from keras.datasets import cifar100
import matplotlib.pyplot as plt
import cv2

# centre the data
def centre_data(train, validation, test):
    
    # calculate the means for each attribute of the training data
    column_means = np.mean(train, axis=0) 
    
    # centre training data by subtracting training data attribute means
    for i in range(len(train)):
        train[i] = train[i] - column_means
    
    # centre testing data by subtracting training data attribute means
    for x in range(len(test)):
        test[x] = test[x] - column_means
        
    for x in range(len(validation)):
        validation[x] = validation[x] - column_means
        
    return train, test, validation

# apply PCA on the data 
def PCA(variance_target, training_data, validation_data, testing_data):

    U, sigma, Vt = np.linalg.svd(training_data, full_matrices=False)
    
    sum_square_singular = np.sum(sigma**2)
    
    ratios = sigma**2/sum_square_singular
    n_components = 0
    explained_variance = 0
    
    # determine how many principle components must be retained to maintain the target level of explained variance
    for i in range(len(ratios)):
        if explained_variance >= variance_target:
            break
        else: 
            n_components += 1
            explained_variance += ratios[i]
    
    return training_data.dot(Vt.T[:, :n_components]), testing_data.dot(Vt.T[:, :n_components]), validation_data.dot(Vt.T[:, :n_components])


# calculate gradient, use L2 regularisation 
def calculate_gradient(weights, regularisation_param, training_data, training_labels, num_classes):
    
    num_training_samples, num_training_features = training_data.shape
    
    gradient = np.zeros(num_training_features, num_classes)
    
    # w^Tx
    linear_output = np.dot(training_data, weights)

    #linear output with labels 
    linear_output_y_i = linear_output[np.arange(num_training_samples),training_labels]
  # distance of point from separating hyperplane?
    # np.newaxis here makes it a column vector 
    # calculate distance?
    
    # distances = y * (np.dot(X, w)) - 1
    delta = linear_output - linear_output_y_i[:,np.newaxis] + 1
    
    ones_and_zeros = np.zeros(delta.shape)
    
    # makes all the places where delta > 0, 1 else 0
    # With lagrange multiplier considered, if the sample is on the support vector: 𝛼 = 1
    # else: 𝛼 = 0
    ones_and_zeros = np.where(delta > 0, 1, 0)
    
    # calculate the sum of each row 
    sum_of_each_row = np.sum(ones_and_zeros, axis=1)
    
    ones_and_zeros[np.arange(num_training_samples), training_labels] = - sum_of_each_row

    gradient = (1/num_training_samples) * np.dot((training_data.T), ones_and_zeros)
    
    # controls the influence of each individual support vector on the objective function. 
    # Greater C decreases the effect of |w|²/2, and results in the narrower margin
    gradient = gradient + (2* regularisation_param * weights)
    
    return gradient 

# train model using stochastic gradient descent 
def train_model(training_data, training_labels, weights, learning_rate, regularisation_param, iterations, batch_size, num_classes):
  
  # number of examples in each batch
  #batch_size = 200
    num_training_samples = len(training_data)
    weights = weights
    for i in range(iterations):
    # create batch
        batch = np.random.choice(num_training_samples, batch_size)
        gradient = calculate_gradient(weights, regularisation_param, training_data[batch], training_labels[batch], num_classes)
        weights = weights - learning_rate * gradient
    return weights

# calculate accuracy of model 
def calculate_accuracy (data, labels, weights):
    accuracy = 0

    prediction = np.zeros(len(data))

  #w^Tx
    linear_output = np.dot(data, weights)

  # returns the indices of the maximum values along an axis, ie. in this case will return the 
  # column index corresponding to the greatest index of each row
    prediction = np.argmax(linear_output, axis=1)

  # count the number of predictions that are correct 
    total_correct_predictions = (prediction == labels).sum()
    num_data_points = len(data)

    accuracy = (total_correct_predictions/num_data_points)*100

    return accuracy


startTime = time.time()

def increase_image_contrast(image):
    xp = [0, 64, 128, 192, 255]
    fp = [0, 16, 128, 240, 255]
    x = np.arange(256)
    table = np.interp(x, xp, fp).astype('uint8')
    return cv2.LUT(image, table)

def increase_all_contrast(train, test):
    new_train = np.copy(train)
    new_test = np.copy(test)
    for i in range(len(train)):
        new_train[i] = increase_image_contrast(train[i])
    for i in range(len(test)):
        new_test[i] = increase_image_contrast(test[i])
    return new_train, new_test

# helper function for concatenating labels onto their corresponding data points
def concatenate_data(training_data, training_labels):
    return np.column_stack((training_data, training_labels))

# data set is randomised and then split in a 70:30 ratio for training:validation sets
def split_into_validation_training(training_matrix):
    
    import random
    random.shuffle(training_matrix)

    training_set = training_matrix[:int(len(training_matrix)*0.7)]
    validation_set = training_matrix[int(len(training_matrix)*0.7):]
    
    return training_set, validation_set

def load_in_dataset_and_preprocess(explained_variance):
  
    (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))
    
    # reshape the data 
    training_data = training_data.reshape(50000, 3072)
    testing_data = testing_data.reshape(10000, 3072)

    concatenated_training = concatenate_data(training_data, training_labels)

    training_set, validation_set = split_into_validation_training(concatenated_training)

    training_data = training_set[:, :-1]
    training_labels = np.squeeze(training_set[:, -1])

    validation_data = validation_set[:, :-1]
    validation_labels = np.squeeze(validation_set[:, -1])

    training_data = training_data.astype('float32')
    testing_data = testing_data.astype('float32')
    validation_data = validation_data.astype('float32')

    # Centre data
    training_data, testing_data, validation_data = centre_data(training_data, testing_data, validation_data)

    # Apply PCA
    training_data, testing_data, validation_data = PCA(explained_variance, training_data, testing_data, validation_data)

    number_training_samples = len(training_data)
    number_validation_samples = len(validation_data)
    number_testing_samples = len(testing_data)

    # Reshape data from channel to rows
    training_data = np.reshape(training_data, (number_training_samples, -1))
    validation_data = np.reshape(validation_data, (number_validation_samples, -1))
    testing_data = np.reshape(testing_data, (number_testing_samples, -1))

    return training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels


# Testing 

In [6]:
import time
import numpy as np
import numpy as np
from keras.datasets import cifar100
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

**Testing weight parameter**

In [5]:
import tensorflow as tf
import time
import csv

def nano_to_seconds(nanoseconds):
    """Converts nanoseconds to seconds rounded to the nearest 5 decimal places.
    
    Parameters
    ----------
    nanoseconds : int
        The nanoseconds to convert
    """

    return np.round((nanoseconds / 1e+9), 5)

**Testing learning rate**

In [10]:
import csv 
def test_svm_learning_rate():
    
    training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)
    
    num_classes = np.max(training_labels) + 1

    learning_rates = [0.00000001]
    
    weights = np.ones((len(training_data[0]), num_classes))

    with open('svm_learning_rates.csv', mode='w', newline='') as csv_file:
    
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['Weights', 'Learning Rate', 'Regularisation Parameters', 'Iterations', 'Batch Size', 'Average Accuracy', 'Average Runtime'])
        reg_param= 50000
        iterations= 15000
        batch_size=200

        for n_learning_rate in learning_rates:
                num_tests = 0
                total_accuracy = 0
                total_runtime = 0

                for i in range(2):
                    start_time = time.time()
                    res_weights = train_model(training_data, training_labels, weights, n_learning_rate, reg_param, iterations,batch_size, num_classes)
                    total_accuracy += calculate_accuracy(training_data, training_labels, res_weights)
                    print(total_accuracy)
                    training_time = time.time() - startTime
                    total_runtime += nano_to_seconds(training_time)
                    num_tests += 1

                avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
                avg_runtime = np.round(total_runtime / float(num_tests), 5)
                result_writer.writerow([weights, n_learning_rate, reg_param, iterations, batch_size, avg_accuracy, avg_runtime])

test_svm_learning_rate()

23.231428571428573


**Testing regularisation parameter**

**Testing iterations**

**Testing batch size**